In [3]:
#resources and credits


In [6]:
#install
pip install transformers
pip install tensorflow
# Install Flask and other required libraries
pip install flask

SyntaxError: invalid syntax (3872544344.py, line 2)

In [5]:
#import
import pdfplumber
import requests
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'pdfplumber'

In [7]:
#function for training


def fetch_text_from_pdf(pdf_link):
    try:
        response = requests.get(pdf_link)
        with open('temp.pdf', 'wb') as file:
            file.write(response.content)
        with pdfplumber.open('temp.pdf') as pdf:
            text = ''
            for page in pdf.pages:
                text += page.extract_text() or ''
        return text
    except Exception as e:
        print(f"Error fetching {pdf_link}: {e}")
        return ""

Training adn testing

In [8]:
nltk.download('punkt')
nltk.download('stopwords')

def preprocess_text(text):
    text = text.encode('ascii', 'ignore').decode('utf-8')
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    custom_stopwords = {'chanakya', 'neeti', 'chapter'}
    stop_words.update(custom_stopwords)
    
    tokens = [token for token in tokens if token not in stop_words]
    
    return " ".join(tokens)

# Test preprocessing on a sample text
sample_text = """
Chanakya Neeti, Chapter 1:
A person should not be too honest.
"""
processed_text = preprocess_text(sample_text)
print(processed_text)

NameError: name 'nltk' is not defined

Collecting and Preprocessing Data from Multiple Sources

In [9]:
corpus = []
books = [
    {"title": "Chanakya Neeti", "pdf_link": "<URL>"},
    {"title": "Chanakya Neeti Darpan", "pdf_link": "<URL>"},
]

for book in books:
    pdf_link = book["pdf_link"]
    text = fetch_text_from_pdf(pdf_link)
    if text:
        processed_text = preprocess_text(text)
        corpus.append(processed_text)

for i, sample in enumerate(corpus[:3], 1):
    print(f"Sample {i}: {sample[:500]}...\n")

Error fetching <URL>: name 'requests' is not defined
Error fetching <URL>: name 'requests' is not defined


Training the Custom LLM

In [10]:
1. Prepare Training Data

SyntaxError: invalid syntax (265257713.py, line 1)

In [11]:
f

NameError: name 'f' is not defined

Define LLM Architecture

In [12]:
mport tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming corpus is your list of preprocessed texts
X = [text.split() for text in corpus]  # Tokenized input texts
y = [text[1:] for text in X]  # Target output (shifted input for training)

# Convert words to integer indices
word_index = {word: idx + 1 for idx, word in enumerate(set([word for text in X for word in text]))}
X = [[word_index[word] for word in text] for text in X]

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Set hyperparameters
vocab_size = len(word_index) + 1  # Plus one for padding
embedding_dim = 128
max_seq_length = 50
lstm_units = 256
output_units = vocab_size

# Pad sequences to ensure uniform input size
X_train = pad_sequences(X_train, maxlen=max_seq_length)
X_val = pad_sequences(X_val, maxlen=max_seq_length)



SyntaxError: invalid syntax (76599491.py, line 1)

Build Model


In [13]:
# Build the model
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_seq_length),
    LSTM(lstm_units),
    Dense(output_units, activation='softmax')
])



NameError: name 'Sequential' is not defined

#Train and Save Model


In [14]:
# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Save the trained model
model.save('custom_llm_model.h5')

NameError: name 'model' is not defined

Evaluation and Testing
Evaluation Metrics

    Perplexity: This metric measures how well the model’s predicted probability distribution matches the actual distribution of words. A lower perplexity indicates better performance.
    BLEU Score: The BLEU (Bilingual Evaluation Understudy) score evaluates the quality of generated text by comparing it to reference texts. Higher scores suggest closer alignment to the reference output. Note that BLEU has limitations and may not capture the full nuance of human-generated responses.
    
Testing the Model

    Generate Text Samples: Use the trained custom LLM to generate text based on a given input prompt.
    Manual Review: Assess generated responses for accuracy, coherence, style, and alignment with Chanakya Neeti’s wisdom. While BLEU and perplexity provide quantitative insights, manual review is vital for capturing subtleties.
    Iterate and Fine-Tune: Based on evaluation results, refine the model to enhance performance.
    Additional Testing Strategies: Consider user studies or A/B testing to evaluate user satisfaction and the chatbot’s effectiveness in practical contexts.

In [16]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Function to evaluate the model
def evaluate_model(input_text, reference_text, max_length=50, num_return_sequences=5):
    try:
        # Load the fine-tuned LLM and tokenizer
        model = TFGPT2LMHeadModel.from_pretrained("./fine_tuned_llm")
        tokenizer = TFGPT2LMHeadModel.from_pretrained("gpt2")

        # Generate text samples
        input_ids = tokenizer.encode(input_text, return_tensors="tf")
        output_ids = model.generate(input_ids, max_length=max_length, num_return_sequences=num_return_sequences)

        # Decode generated text
        generated_text = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]
        logging.info("Generated Text: %s", generated_text)

        # Calculate BLEU score
        bleu_scores = [sentence_bleu([ref.split()], gen.split()) for ref, gen in zip(reference_text, generated_text)]

        # Calculate perplexity
        input_ids = tokenizer.encode(reference_text[0], return_tensors="tf")
        logits = model(input_ids)[0]
        cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        perplexity = np.exp(cross_entropy(input_ids, logits).numpy())

        return generated_text, bleu_scores, perplexity

    except Exception as e:
        logging.error("An error occurred during evaluation: %s", e)
        return None, None, None


ModuleNotFoundError: No module named 'tensorflow'

Example usage

In [17]:

# Example usage
input_text = "What is the meaning of life?"
reference_text = ["The meaning of life is to be happy."]
generated_text, bleu_scores, perplexity = evaluate_model(input_text, reference_text)

if generated_text is not None:
    print("Generated Text:")
    for text in generated_text:
        print(text)
    print("BLEU Scores:", bleu_scores)
    print("Perplexity:", perplexity)

NameError: name 'evaluate_model' is not defined

Fine-tuning and Optimization

To fine-tune our custom Large Language Model (LLM), we load a pre-trained model like GPT-2 and unfreeze the last few layers for training. This process allows the model to adapt while retaining its foundational knowledge.
Steps for Fine-tuning

    Load the Pre-trained Model: Use the transformers library to load GPT-2 as the base model.
    Unfreeze Layers: Unfreeze the last few layers to enable fine-tuning on the custom dataset.
    Define the Optimizer: Choose the Adam optimizer with a learning rate of 1e−51e-51e−5. Note that this may need adjustment based on experimentation and dataset characteristics.
    Compile the Model: Compile with sparse categorical cross-entropy as the loss function.
    Train the Model: Train the model on the prepared datasets.
    Save the Fine-tuned Model: Save the model for future use.

In [18]:
import tensorflow as tf
from transformers import TFGPT2LMHeadModel

# Load the pre-trained GPT-2 model
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

# Unfreeze specific layers for fine-tuning
for layer in model.layers[-6:]:
    layer.trainable = True

# Define the optimizer and learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)  # Learning rate may require tuning

# Compile the model
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Train the model on the custom dataset
model.fit(train_dataset, epochs=10, validation_data=val_dataset)

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_llm")

ModuleNotFoundError: No module named 'tensorflow'

Deployment and User Interface

This section will explore methods for deploying our fine-tuned LLM and creating a user interface to interact with it. We’ll utilize Next.js, TypeScript, and Google Material UI for the front end, while Python and Flask for the back end.

Backend (Python and Flask) — app.py:

In [19]:
from flask import Flask, request, jsonify
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

app = Flask(__name__)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = TFGPT2LMHeadModel.from_pretrained("gpt2")

@app.route("/chatbot", methods=["POST"])
def chatbot():
    message = request.json["message"]
    input_ids = tokenizer.encode(message, return_tensors="tf")
    output_ids = model.generate(input_ids, max_length=100, num_return_sequences=1)
    response = tokenizer.decode(output_ids[0])

    return jsonify({"response": response})

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

ModuleNotFoundError: No module named 'flask'

Frontend (Next.js, Material UI, and TypeScript) — pages/index.tsx: